## Data ：
- Raw Text

## Models：
-  Logistic Regression
-  Naíve Bayes
-  Decision Tree

In [1]:
import os
os.getcwd()

'/Users/loewi/Documents/GitHub/text-formality-classifier'

In [2]:
from time import time

t0 = time() 

path1 = 'data/formal.txt'
with open(path1) as f:
    corpus_formal = f.readlines()
print("fertig")

path2 = 'data/informal.txt'
with open(path2) as f:
    corpus_informal = f.readlines()
print("fertig")
#%%
from sklearn.utils import shuffle
import numpy as np

X = np.array(corpus_formal+corpus_informal)
y = np.array([0]*len(corpus_formal) + [1]*len(corpus_informal) )

X, y = shuffle(X,y)

from sklearn.model_selection import train_test_split

X_t, X_te, y_t, y_te = train_test_split(
                X, y, test_size=0.2, random_state=0)
#%%'
all_train = dict(classes=[0, 1], #formal 0, informal 1
                data = X_t,
                categories=np.array(y_t))

all_test = dict(classes=[0, 1], #formal 0, informal 1
                data = X_te,
                categories=np.array(y_te))

duration = time() - t0
print('%0.2fs get data package ：）'%duration)

X_train,X_test = all_train['data'], all_test['data'] #list of strings
y_train, y_test =all_train['categories'], all_test['categories'] #array
print('X & y prepared')

fertig
fertig
8.13s get data package ：）
X & y prepared


In [10]:
from feature import features
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,HashingVectorizer,TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score,StratifiedKFold,GridSearchCV,RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from pprint import pprint
from joblib import dump

# Logistic Regression

In [9]:
#fine tune
pipeline = Pipeline([
                ('vect', HashingVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression()),
                ])        
parameters = {
    'clf__solver': ('newton-cg', 'lbfgs','sag'),  
    'clf__multi_class':('ovr','multinomial'),  
    }   

cv = StratifiedKFold(n_splits= 5 ,random_state=0,shuffle=True)
clf = GridSearchCV(pipeline,parameters,n_jobs=-1, verbose=1,cv = cv)

print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
pprint(parameters)

t0 = time()
clf.fit(X_train, y_train)
m, s = divmod(time() - t0, 60)
h, m = divmod(m, 60)
print ("done in %02d:%02d:%02d" % (h, m, s))
print()

print("Best score: %0.3f" % clf.best_score_)
print("Best parameters set:")
best_parameters = clf.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name])) 
       

Performing grid search...
pipeline: ['vect', 'tfidf', 'clf']
parameters:
{'clf__multi_class': ('ovr', 'multinomial'),
 'clf__solver': ('newton-cg', 'lbfgs', 'sag')}
Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  7.6min finished


done in 00:07:49

Best score: 0.754
Best parameters set:
	clf__multi_class: 'ovr'
	clf__solver: 'newton-cg'


In [9]:
#predict
from sklearn import metrics  
pipeline_validated = Pipeline([
                ('vect', HashingVectorizer()),

                ('clf', LogisticRegression(
                        solver='newton-cg',  
                        multi_class='ovr', 
                        )),
                ])    
print('Training...') 
pipeline_validated.fit(X_train, y_train) 

pred = pipeline_validated.predict(X_test)
score = metrics.accuracy_score(y_test, pred)
print("testing accuracy:   %0.3f" % score)


Training...
testing accuracy:   0.753


In [12]:
path = "model/raw_lg.jbl"
dump(pipeline_validated, path)

['model/raw_lg.jbl']

# Naíve Bayes

In [15]:
# fine tune
nbc = Pipeline([
                ('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB()),
                ])
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__max_features': (None, 5000, 10000, 20000),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
#    'tfidf__use_idf': (True, False),
#    'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': (0.1, 0.01, 0.001, 0.0001),
}    
cv = StratifiedKFold(n_splits= 5 ,random_state=0,shuffle=True)
clf = GridSearchCV(nbc,parameters,n_jobs=-1, verbose=1, cv = cv)
print("Performing grid search...")
print("pipeline:", [name for name, _ in nbc.steps])
print("parameters:")
pprint(parameters)
t0 = time()
clf.fit(X_train, y_train)
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % clf.best_score_)
print("Best parameters set:")
best_parameters = clf.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))        
    

Performing grid search...
pipeline: ['vect', 'tfidf', 'clf']
parameters:
{'clf__alpha': (0.1, 0.01, 0.001, 0.0001),
 'vect__max_df': (0.5, 0.75, 1.0),
 'vect__max_features': (None, 5000, 10000, 20000),
 'vect__ngram_range': ((1, 1), (1, 2))}
Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 28.4min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed: 67.2min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed: 74.0min finished


done in 4446.348s

Best score: 0.696
Best parameters set:
	clf__alpha: 0.1
	vect__max_df: 0.5
	vect__max_features: 5000
	vect__ngram_range: (1, 2)


In [13]:
#predict   
from sklearn import metrics  
clf_validated = Pipeline([
                ('vect', CountVectorizer(
                        max_df = 0.5,
                        max_features = 5000,
                        ngram_range = (1, 2),
                        )),
                ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB(alpha = 0.1)),
                ])    
                
clf_validated.fit(X_train, y_train)  

cv = StratifiedKFold(n_splits= 5 ,random_state=0,shuffle=True)
scores = cross_val_score(clf_validated, X_train, y_train,  cv = cv, scoring='accuracy')
print (scores)
score = scores.mean()
print("training accuracy: %0.3f (+/- %0.3f)" % (score, scores.std() * 2))
              
pred = clf_validated.predict(X_test)
score = metrics.accuracy_score(y_test, pred)
print("testing accuracy:   %0.3f" % score)

[0.74994023 0.75188285 0.75080693 0.74853556 0.74616695]
training accuracy: 0.749 (+/- 0.004)
testing accuracy:   0.746


In [14]:
path = "model/raw_nb.jbl"
dump(clf_validated, path)

['model/raw_nb.jbl']

# Decision Tree

In [8]:
#fine tune
pipeline = Pipeline([
                ('vect', HashingVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', DecisionTreeClassifier()),
                ])
parameters = {
            'clf__max_depth':(100, 800),
            'clf__max_leaf_nodes':(30, 300, 3000),
            'clf__min_samples_leaf':(1, 2),
            'clf__max_features': ('auto',20000),
            'clf__min_samples_split': (0.5, 1.0)
}    

cv = StratifiedKFold(n_splits= 5 ,random_state=0,shuffle=True)
clf = GridSearchCV(pipeline, parameters,n_jobs=-1, verbose=1, cv = cv)
print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
pprint(parameters)
t0 = time()
clf.fit(X_train, y_train)
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % clf.best_score_)
print("Best parameters set:")
best_parameters = clf.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))   

Performing grid search...
pipeline: ['vect', 'tfidf', 'clf']
parameters:
{'clf__max_depth': (100, 800),
 'clf__max_features': ('auto', 20000),
 'clf__max_leaf_nodes': (30, 300, 3000),
 'clf__min_samples_leaf': (1, 2),
 'clf__min_samples_split': (0.5, 1.0)}
Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 28.1min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed: 35.9min finished


done in 2159.162s

Best score: 0.606
Best parameters set:
	clf__max_depth: 800
	clf__max_features: 20000
	clf__max_leaf_nodes: 30
	clf__min_samples_leaf: 2
	clf__min_samples_split: 0.5


In [15]:
# predict
from sklearn import metrics  
clf_validated = Pipeline([
                ('vect', HashingVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', DecisionTreeClassifier(
                        
                        max_features = 20000,
                        max_depth = 800,
                        min_samples_split = 2,
                        min_samples_leaf = 0.5,
                        max_leaf_nodes = 30,
        
                        )),
                ]) 
print('Fitting...')                
clf_validated.fit(X_train, y_train)  

cv = StratifiedKFold(n_splits= 5 ,random_state=0,shuffle=True)
scores = cross_val_score(clf_validated, X_train, y_train,  cv = cv, scoring='accuracy')
print (scores)
score = scores.mean()
print("training accuracy: %0.3f (+/- %0.3f)" % (score, scores.std() * 2))
pred = clf_validated.predict(X_test)
score = metrics.accuracy_score(y_test, pred)
print("testing accuracy:   %0.3f" % score)

Fitting...
[0.50005977 0.50005977 0.50005977 0.50005977 0.50007472]
training accuracy: 0.500 (+/- 0.000)
testing accuracy:   0.500


In [16]:
path = "model/raw_dt.jbl"
dump(clf_validated, path)

['model/raw_dt.jbl']